In [2]:
import napari
from skimage.data import astronaut
from napari_picasso.main_widget import PicassoWidget
from pyseq import image_analysis as ia

#image_path = 'Z:\\gpfs\\commons\\groups\\nygcfaculty\\PySeq\\20210323_4i4color\\zarrs\\m4_o25933_labels.zarr'
image_path = 'Z:\\gpfs\\commons\\groups\\nygcfaculty\\PySeq\\20210323_4i4color\\zarrs\\m4.zarr'
o = 25933
im = ia.get_HiSeqImages(image_path)
cropped_im = im.im.sel(row = slice(3800,4000), col = slice(4100, 4300), cycle=5, obj_step=o)


viewer = napari.Viewer()
#widget = PicassoWidget(viewer)
widget = PicassoWidget(viewer)
viewer.window.add_dock_widget(widget)
#viewer.add_image(astronaut(), channel_axis=-1)
viewer.add_image(cropped_im.sel(channel=558).compute(), name='Laminin1b', blending = 'additive', contrast_limits = (0,4095))
viewer.add_image(cropped_im.sel(channel=610).compute(), name='GFAP', blending = 'additive',  contrast_limits = (0,4095))
# viewer.add_image(im.im.sel(obj_step = o, channel=687, cycle=5), name='MBP', blending = 'additive')
# viewer.add_image(im.im.sel(obj_step = o, channel=740, cycle=5), name='ELAVL2', blending = 'additive')


c:\users\kpandit\pyseq2500\pyseq\image_analysis.py:955: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  im = xr.open_zarr(fn).to_array()


ImageAnalysis::Opened m4 


<Image layer 'GFAP' at 0x1b0ac1f2c10>

C:\Users\kpandit\.conda\envs\cuda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda
available memory 2.00 GiB
number of images 2
6704.3328
It 10 - total loss: 0.8100370764732361, total MI loss: 0.8099607825279236, total contrast loss: 7.631182961631566e-05
It 20 - total loss: 0.8100892901420593, total MI loss: 0.810013473033905, total contrast loss: 7.582903344882652e-05
It 30 - total loss: 0.8101129531860352, total MI loss: 0.8100371956825256, total contrast loss: 7.57634625188075e-05
It 40 - total loss: 0.8100488185882568, total MI loss: 0.8099733591079712, total contrast loss: 7.548928260803223e-05
It 50 - total loss: 0.8099384903907776, total MI loss: 0.8098636269569397, total contrast loss: 7.483959052478895e-05
It 60 - total loss: 0.8099039793014526, total MI loss: 0.8098304271697998, total contrast loss: 7.355213165283203e-05
It 70 - total loss: 0.8098901510238647, total MI loss: 0.8098180294036865, total contrast loss: 7.212162017822266e-05
It 80 - total loss: 0.809851884841919, total MI loss: 0.8097811937332153, total contrast loss: 7.0708993007428

In [6]:
import torch
from dask.utils import format_bytes
torch.cuda.memory_reserved() - torch.cuda.memory_allocated()
print(torch.cuda.memory_reserved())
torch.cuda.get_device_properties('cuda')

0


_CudaDeviceProperties(name='Quadro K620', major=5, minor=0, total_memory=2048MB, multi_processor_count=3)

In [2]:
import pynvml

ModuleNotFoundError: No module named 'pynvml'

In [13]:
import napari
import numpy as np
from napari_picasso.main_widget import PicassoWidget


# Make sink source images
s = 512; background = 100; signal = 500; alpha = 0.5

sink = np.random.rand(s,s)*10+background
sink[:100, :100] += np.random.rand(100,100)*2000+signal

source = np.zeros((s,s))
source[-100:,-100:] += np.random.rand(100,100)*2000+signal

sink += alpha*source
source += np.random.rand(s,s)*10+background

# Add to viewer
viewer = napari.Viewer(show=False)
viewer.add_image(sink, name='sink')
viewer.add_image(source, name='source')

# setup widget
widget = PicassoWidget(viewer)
widget.add_sink_widget()

sink = widget['sink0']
sink.sink_list.bind(sink)
sink.show_sources()
sink.update_mixing_params()

for s in widget.sinks:
    sink = widget[f'sink{s}']

In [14]:
import time
from napari_picasso.utils import get_image_layers

widget.make_model(max_iter = 100)
assert widget._progress.total == 100

# start = time.time()
# while time.time() <= start+30:
#     print(widget._worker.is_running)
#     if widget.picasso_params is not None:
#         break

image_layers = get_image_layers(widget._viewer)
image_names = [l.name for l in image_layers]
image_names


Using cuda


['sink', 'source']

model sinks 1
model images 2
n images 2
Start training

It 10 - total loss: 0.8104085326194763, total MI loss: 0.809951663017273, total contrast loss: 0.00045687556848861277
It 20 - total loss: 0.8103855848312378, total MI loss: 0.8099296689033508, total contrast loss: 0.0004559338267426938
It 30 - total loss: 0.8103542923927307, total MI loss: 0.8098993301391602, total contrast loss: 0.000454956287285313


In [15]:
start = time.time()
while time.time() <= start+30:
    if widget.picasso_params is not None:
        break
image_layers = get_image_layers(widget._viewer)
image_names = [l.name for l in image_layers]
print(image_names)

['sink', 'source']
Finished training



In [18]:
image_layers = get_image_layers(widget._viewer)
image_names = [l.name for l in image_layers]
print(image_names)

['sink', 'source']


In [2]:
from napari.utils import progress
from time import sleep
from napari.qt.threading import thread_worker


pbr = progress(range(10))
def pbr_update(i):
    pbr.update(i)
    if i == 9:
        pbr.close()
    print(f'from yield_value {i}')
    
@thread_worker(connect={"yielded": pbr_update})
def long_running(steps=10, repeats=4, delay=1):
    for i in pbr:
        print(i)
        yield(i)
        sleep(delay)


    
worker = long_running()


# steps = 10
# pbr = progress(range(steps))
# for i in long_running(steps):
#     print(i)
#     pbr.update(i)
# pbr.close()
    


0
from yield_value 0
1
from yield_value 1
2
from yield_value 2
3
from yield_value 3
4
from yield_value 4
5
from yield_value 5
6
from yield_value 6
7
from yield_value 7
8
from yield_value 8
9
from yield_value 9
Using cpu
It 10 - total loss: 0.8252242207527161, total MI loss: 0.8099648952484131, total contrast loss: 0.01525931991636753
It 20 - total loss: 0.8239518404006958, total MI loss: 0.8086955547332764, total contrast loss: 0.015256292186677456
It 30 - total loss: 0.822663426399231, total MI loss: 0.8074032068252563, total contrast loss: 0.015260196290910244
It 40 - total loss: 0.8212747573852539, total MI loss: 0.8060032725334167, total contrast loss: 0.01527147926390171
It 50 - total loss: 0.8192973732948303, total MI loss: 0.8040131330490112, total contrast loss: 0.015284234657883644
It 60 - total loss: 0.8171197175979614, total MI loss: 0.8018242716789246, total contrast loss: 0.0152954226359725
It 70 - total loss: 0.8144541382789612, total MI loss: 0.799152135848999, total con

In [5]:
from napari.utils import progress
pbr = progress(range(5,10))
pbr.total

5

In [10]:
widget.run_btn.bind(True)
widget.run_btn.bind(False)
widget.run_btn.get_value()

False

Using cpu
It 10 - total loss: 1.79609215259552, total MI loss: 1.6190526485443115, total contrast loss: 0.1770394742488861
It 20 - total loss: 1.793644666671753, total MI loss: 1.6173226833343506, total contrast loss: 0.17632201313972473
It 30 - total loss: 1.7902164459228516, total MI loss: 1.6147146224975586, total contrast loss: 0.1755017638206482
It 40 - total loss: 1.7855160236358643, total MI loss: 1.6109461784362793, total contrast loss: 0.17456978559494019
It 50 - total loss: 1.7789340019226074, total MI loss: 1.6054050922393799, total contrast loss: 0.17352886497974396
It 60 - total loss: 1.770835041999817, total MI loss: 1.5984455347061157, total contrast loss: 0.17238953709602356
It 70 - total loss: 1.7589645385742188, total MI loss: 1.5878045558929443, total contrast loss: 0.17115992307662964
It 80 - total loss: 1.7408010959625244, total MI loss: 1.570955514907837, total contrast loss: 0.16984562575817108
It 90 - total loss: 1.7149162292480469, total MI loss: 1.546468615531

In [4]:
widget['run_btn']

AttributeError: 'PicassoWidget' object has no attribute 'run_btn'

In [6]:
test = widget.train_model()
test.start()

training
in for loop
It 10 - total loss: 1.7959944009780884, total MI loss: 1.6189546585083008, total contrast loss: 0.1770397126674652
It 20 - total loss: 1.7938904762268066, total MI loss: 1.617567539215088, total contrast loss: 0.17632290720939636
It 30 - total loss: 1.79129159450531, total MI loss: 1.6157886981964111, total contrast loss: 0.17550288140773773
It 40 - total loss: 1.787611722946167, total MI loss: 1.6130425930023193, total contrast loss: 0.17456918954849243
It 50 - total loss: 1.7816733121871948, total MI loss: 1.6081480979919434, total contrast loss: 0.17352516949176788
It 60 - total loss: 1.7726953029632568, total MI loss: 1.6003142595291138, total contrast loss: 0.17238101363182068
It 70 - total loss: 1.7594220638275146, total MI loss: 1.5882782936096191, total contrast loss: 0.1711438000202179
It 80 - total loss: 1.739220380783081, total MI loss: 1.5694035291671753, total contrast loss: 0.16981683671474457
It 90 - total loss: 1.709040641784668, total MI loss: 1.54

In [7]:
def out_test():
    i = 1
    def in_test():
        print(i)
    in_test()
        
        
out_test()

1


In [15]:
np.where(mm[0,:,i] == 1)

(array([1], dtype=int64),)

In [14]:
np.where(mm[0,:,i] == 1)[0][0]

1

In [21]:
viewer.layers[0].as_layer_data_tuple()[2]

'image'

In [11]:
isinstance(viewer.layers[0], 'napari.layers.image.image.Image')

TypeError: isinstance() arg 2 must be a type or tuple of types

In [18]:
import numpy as np
napari.layers.Layer(np.zeros((10,10)))

TypeError: Can't instantiate abstract class Layer with abstract methods _extent_data, _get_ndim, _get_state, _get_value, _set_view_slice, _update_thumbnail, data